# Time Series Forecasting for E-Commerce Sales with Behavioral Clustering

In [ ]:
!pip -q install pandas numpy matplotlib seaborn scikit-learn tensorflow openpyxl pytrends pmdarima statsmodels scipy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.stats import skew, kurtosis, probplot
from statsmodels.stats.diagnostic import acorr_ljungbox
from pmdarima import auto_arima
from pytrends.request import TrendReq
import warnings
warnings.filterwarnings('ignore')

## Dataset Acquisition and Initial Processing

In [ ]:
dataset_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
retail_data = pd.read_excel(dataset_url)

retail_data.dropna(inplace=True)
retail_data = retail_data[~retail_data['InvoiceNo'].astype(str).str.startswith('C')]
retail_data['Revenue'] = retail_data['Quantity'] * retail_data['UnitPrice']
retail_data['InvoiceDate'] = pd.to_datetime(retail_data['InvoiceDate'])

sales_by_day = retail_data.set_index('InvoiceDate').resample('D')['Revenue'].sum()
complete_date_range = pd.date_range(start=sales_by_day.index.min(), end=sales_by_day.index.max())
sales_by_day = sales_by_day.reindex(complete_date_range, fill_value=0)
sales_by_day.index.name = 'Date'

print(f"Original dataset dimensions: {retail_data.shape}")
print(f"Time series length: {sales_by_day.shape}")

## Visual Exploration of Sales Patterns

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
sales_by_day.plot(ax=ax, color='steelblue', linewidth=1.5)
ax.set_title("Revenue Trends Across Time Horizon", fontsize=15, weight='bold')
ax.set_xlabel("Timeline", fontsize=12)
ax.set_ylabel("Revenue (£)", fontsize=12)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## Stationarity Assessment and Temporal Decomposition

In [ ]:
adf_test = adfuller(sales_by_day)
print(f"Augmented Dickey-Fuller Statistic: {adf_test[0]:.4f}")
print(f"Statistical Significance (p): {adf_test[1]:.4f}")

if adf_test[1] > 0.05:
    print("Series exhibits non-stationarity characteristics")
else:
    print("Series demonstrates stationarity")

temporal_components = seasonal_decompose(sales_by_day, model='additive', period=30)
fig = temporal_components.plot()
fig.set_size_inches(14, 8)
plt.tight_layout()
plt.show()

## Correlation Structure Analysis

In [ ]:
log_transformed = np.log1p(sales_by_day)
differenced_series = log_transformed.diff().dropna()

pre_transform_pval = adfuller(sales_by_day)[1]
post_transform_pval = adfuller(differenced_series)[1]
print(f"Pre-transformation p-value: {pre_transform_pval:.4f}")
print(f"Post-transformation p-value: {post_transform_pval:.4f}")

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

plot_acf(differenced_series, lags=60, ax=axes[0], alpha=0.05)
axes[0].set_title("Autocorrelation Function", fontsize=13, weight='bold')
axes[0].set_xlabel("Lag Period")
axes[0].set_ylabel("Correlation Coefficient")
axes[0].grid(True, alpha=0.3)

plot_pacf(differenced_series, lags=60, ax=axes[1], method='ywm', alpha=0.05)
axes[1].set_title("Partial Autocorrelation Function", fontsize=13, weight='bold')
axes[1].set_xlabel("Lag Period")
axes[1].set_ylabel("Partial Correlation")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Anomaly Identification and Data Sanitization

In [ ]:
rolling_window = 30
percentile_25 = sales_by_day.rolling(rolling_window, center=True).quantile(0.25)
percentile_75 = sales_by_day.rolling(rolling_window, center=True).quantile(0.75)
interquartile_range = percentile_75 - percentile_25

threshold_lower = percentile_25 - 1.5 * interquartile_range
threshold_upper = percentile_75 + 1.5 * interquartile_range
iqr_anomalies = (sales_by_day < threshold_lower) | (sales_by_day > threshold_upper)

standardized_scores = (sales_by_day - sales_by_day.mean()) / sales_by_day.std()
zscore_anomalies = np.abs(standardized_scores) > 3

combined_anomalies = iqr_anomalies | zscore_anomalies

sanitized_series = sales_by_day.copy()
median_imputation = sales_by_day.rolling(window=7, center=True).median()
sanitized_series[combined_anomalies] = median_imputation[combined_anomalies]

fig, ax = plt.subplots(figsize=(15, 6))
ax.plot(sales_by_day, label='Raw Data', alpha=0.5, color='lightgray', linewidth=1)
ax.plot(sanitized_series, label='Sanitized Data', linewidth=2, color='darkblue')
ax.scatter(sales_by_day[combined_anomalies].index, sales_by_day[combined_anomalies].values,
           color='crimson', s=60, label='Detected Anomalies', zorder=5)
ax.set_title("Hybrid Anomaly Detection with Median Imputation", fontsize=14, weight='bold')
ax.set_xlabel("Date")
ax.set_ylabel("Revenue")
ax.legend(loc='upper left')
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## Training and Validation Partitioning

In [ ]:
split_ratio = 0.9
split_index = int(len(sales_by_day) * split_ratio)
training_set, validation_set = sales_by_day[:split_index], sales_by_day[split_index:]

print(f"Training observations: {len(training_set)}")
print(f"Validation observations: {len(validation_set)}")

## Holt-Winters Exponential Smoothing Implementation

In [ ]:
hw_model = ExponentialSmoothing(training_set, seasonal='add', seasonal_periods=30)
hw_fitted = hw_model.fit()
hw_predictions = hw_fitted.forecast(len(validation_set))

hw_mae = mean_absolute_error(validation_set, hw_predictions)
hw_rmse = np.sqrt(mean_squared_error(validation_set, hw_predictions))
hw_r2 = r2_score(validation_set, hw_predictions)

print(f"Mean Absolute Error: {hw_mae:.2f}")
print(f"Root Mean Squared Error: {hw_rmse:.2f}")
print(f"Coefficient of Determination: {hw_r2:.4f}")

## Automated ARIMA Parameter Selection

In [ ]:
auto_model = auto_arima(
    training_set,
    start_p=1, start_q=1,
    max_p=5, max_q=5,
    d=None,
    seasonal=False,
    stepwise=True,
    suppress_warnings=True,
    error_action='ignore',
    trace=True
)

optimal_parameters = auto_model.order
print(f"\nOptimal ARIMA configuration: {optimal_parameters}")

arima_model = ARIMA(training_set, order=optimal_parameters)
arima_fitted = arima_model.fit()
arima_predictions = arima_fitted.forecast(steps=len(validation_set))

print(f"Generated {len(arima_predictions)} forecast points")

## Comparative Performance Evaluation

In [ ]:
def calculate_mape(actual, predicted):
    valid_mask = actual != 0
    if valid_mask.sum() == 0:
        return np.nan
    return np.mean(np.abs((actual[valid_mask] - predicted[valid_mask]) / actual[valid_mask])) * 100

def compute_metrics(actual, predicted, name='Model'):
    mae_score = mean_absolute_error(actual, predicted)
    rmse_score = np.sqrt(mean_squared_error(actual, predicted))
    mape_score = calculate_mape(actual, predicted)
    r2_score_val = r2_score(actual, predicted)
    bias_score = np.mean(predicted - actual)
    
    return {
        'Algorithm': name,
        'MAE': round(mae_score, 2),
        'RMSE': round(rmse_score, 2),
        'MAPE (%)': round(mape_score, 2),
        'R²': round(r2_score_val, 4),
        'Bias': round(bias_score, 2)
    }

performance_metrics = []
performance_metrics.append(compute_metrics(validation_set, hw_predictions, 'Holt-Winters'))
performance_metrics.append(compute_metrics(validation_set, arima_predictions, 'Auto-ARIMA'))

comparison_table = pd.DataFrame(performance_metrics)
print("\n" + "="*70)
print(comparison_table.to_string(index=False))
print("="*70)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 7))

ax.plot(validation_set.index, validation_set, label='Observed Values', 
        color='black', linewidth=2.5, marker='o', markersize=4)
ax.plot(validation_set.index, hw_predictions, label='Holt-Winters Forecast', 
        linestyle='--', marker='s', markersize=5, color='darkorange', alpha=0.8)
ax.plot(validation_set.index, arima_predictions, label='ARIMA Forecast', 
        linestyle='-.', marker='^', markersize=5, color='darkgreen', alpha=0.8)

ax.set_title('Predictive Model Performance Comparison', fontsize=16, weight='bold')
ax.set_xlabel('Time Period', fontsize=13)
ax.set_ylabel('Revenue (£)', fontsize=13)
ax.legend(loc='best', fontsize=11)
ax.grid(alpha=0.4, linestyle=':')
plt.tight_layout()
plt.show()

## Forecast Error Diagnostics

In [ ]:
forecast_errors = validation_set - hw_predictions
error_mean = forecast_errors.mean()
error_std = forecast_errors.std()

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].plot(forecast_errors, color='darkred', linewidth=1.8)
axes[0].axhline(0, color='black', linestyle='--', linewidth=1.2)
axes[0].fill_between(forecast_errors.index, error_mean - 2*error_std, 
                     error_mean + 2*error_std, alpha=0.25, color='gray')
axes[0].set_title('Temporal Error Distribution', fontsize=12, weight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Residual Value')
axes[0].grid(alpha=0.3)

axes[1].hist(forecast_errors, bins=25, edgecolor='black', alpha=0.75, color='salmon')
axes[1].axvline(0, color='darkred', linestyle='--', linewidth=2)
axes[1].set_title('Error Frequency Distribution', fontsize=12, weight='bold')
axes[1].set_xlabel('Residual Magnitude')
axes[1].set_ylabel('Frequency')
axes[1].grid(alpha=0.3)

probplot(forecast_errors, dist="norm", plot=axes[2])
axes[2].set_title('Normality Assessment (Q-Q Plot)', fontsize=12, weight='bold')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

ljung_box_test = acorr_ljungbox(forecast_errors, lags=[10], return_df=True)
print("\nLjung-Box Autocorrelation Test:")
print(ljung_box_test)
print(f"\nError Statistics:")
print(f"Mean: {error_mean:.2f}")
print(f"Standard Deviation: {error_std:.2f}")
print(f"Skewness: {skew(forecast_errors):.2f}")
print(f"Kurtosis: {kurtosis(forecast_errors):.2f}")

## External Trend Data Integration

In [ ]:
trends_api = TrendReq(hl='en-US', tz=360)
search_terms = ['online shopping', 'e-commerce']

period_start = sales_by_day.index.min().strftime('%Y-%m-%d')
period_end = sales_by_day.index.max().strftime('%Y-%m-%d')
query_timeframe = f'{period_start} {period_end}'

trends_api.build_payload(kw_list=search_terms, cat=0, timeframe=query_timeframe, geo='GB', gprop='')
trends_data = trends_api.interest_over_time()

if 'isPartial' in trends_data.columns:
    trends_data = trends_data.drop(columns=['isPartial'])

trends_data.columns = [f"{col.replace(' ', '_').lower()}_index" for col in trends_data.columns]
trends_daily = trends_data.resample('D').ffill()

sales_by_day.index.name = 'Date'
trends_daily.index.name = 'Date'

enriched_dataset = pd.merge(
    sales_by_day.to_frame(name='Revenue'),
    trends_daily,
    left_index=True,
    right_index=True,
    how='left'
)

enriched_dataset.fillna(method='ffill', inplace=True)
enriched_dataset.fillna(method='bfill', inplace=True)
enriched_dataset.fillna(0, inplace=True)

print(f"Enhanced dataset dimensions: {enriched_dataset.shape}")
print(f"\nFirst 5 records:")
print(enriched_dataset.head())

## Behavioral Pattern Segmentation

In [ ]:
feature_engineered = enriched_dataset.copy()

feature_engineered['weekday'] = feature_engineered.index.dayofweek
feature_engineered['month_num'] = feature_engineered.index.month
feature_engineered['quarter_num'] = feature_engineered.index.quarter
feature_engineered['weekend_flag'] = feature_engineered.index.dayofweek.isin([5, 6]).astype(int)
feature_engineered['iso_week'] = feature_engineered.index.isocalendar().week.astype(int)

feature_engineered['ma_7day'] = feature_engineered['Revenue'].rolling(window=7).mean()
feature_engineered['volatility_7day'] = feature_engineered['Revenue'].rolling(window=7).std()

feature_engineered['revenue_lag1'] = feature_engineered['Revenue'].shift(1)
feature_engineered['revenue_lag7'] = feature_engineered['Revenue'].shift(7)
feature_engineered['revenue_lag30'] = feature_engineered['Revenue'].shift(30)

imputation_cols = ['ma_7day', 'volatility_7day', 'revenue_lag1', 'revenue_lag7', 'revenue_lag30']
for col in imputation_cols:
    feature_engineered[col].fillna(feature_engineered[col].mean(), inplace=True)

clustering_features = [
    'online_shopping_index', 'e-commerce_index',
    'weekday', 'month_num', 'quarter_num', 'weekend_flag', 'iso_week',
    'ma_7day', 'volatility_7day',
    'revenue_lag1', 'revenue_lag7', 'revenue_lag30'
]

feature_matrix = feature_engineered[clustering_features]
print(f"Feature matrix shape: {feature_matrix.shape}")

In [ ]:
normalizer = StandardScaler()
normalized_features = normalizer.fit_transform(feature_matrix)

cluster_range = range(2, 12)
inertia_values = []

for n_clusters in cluster_range:
    clustering_model = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    clustering_model.fit(normalized_features)
    inertia_values.append(clustering_model.inertia_)

fig, ax = plt.subplots(figsize=(11, 6))
ax.plot(cluster_range, inertia_values, marker='D', linestyle='-', linewidth=2, markersize=8, color='darkviolet')
ax.set_title('Optimal Cluster Determination via Elbow Method', fontsize=14, weight='bold')
ax.set_xlabel('Number of Clusters', fontsize=12)
ax.set_ylabel('Within-Cluster Sum of Squares', fontsize=12)
ax.set_xticks(cluster_range)
ax.grid(True, alpha=0.4)
plt.tight_layout()
plt.show()

In [ ]:
selected_clusters = 4
final_clustering = KMeans(n_clusters=selected_clusters, random_state=42, n_init=10)
feature_engineered['segment'] = final_clustering.fit_predict(normalized_features)

fig, ax = plt.subplots(figsize=(16, 7))

color_palette = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A']
for segment_id in range(selected_clusters):
    segment_subset = feature_engineered[feature_engineered['segment'] == segment_id]
    ax.scatter(segment_subset.index, segment_subset['Revenue'], 
              label=f'Segment {segment_id}', alpha=0.7, s=50, color=color_palette[segment_id])

ax.set_title('Consumer Behavior Segmentation Analysis', fontsize=16, weight='bold')
ax.set_xlabel('Timeline', fontsize=13)
ax.set_ylabel('Revenue (£)', fontsize=13)
ax.legend(loc='upper left', fontsize=11)
ax.grid(alpha=0.3, linestyle=':')
plt.tight_layout()
plt.show()

print("\nSegment-wise Revenue Statistics:")
print("="*80)
segment_summary = feature_engineered.groupby('segment')['Revenue'].describe()
print(segment_summary)
print("="*80)